<a href="https://colab.research.google.com/github/mcash8/Info-Theory-Project/blob/main/CL%20Learning%20Experiments/Cascade_MNISTSaveActivation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp drive/MyDrive/Cascade/Cascade_calculate_loaded_data_information1.py /content/
!cp drive/MyDrive/Cascade/kde.py /content/

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.io as sio
from keras.layers.core import Dense
import keras
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.models import Model
import tqdm
import time
import keras.backend as K
from time import sleep
import os
import pickle as cPickle
import matplotlib.pyplot as plt
from Cascade_calculate_loaded_data_information1 import calculate_layer_mutual_information,extract_probs,calculate_mutual_information,calculate_lower_mutual_information
import h5py
from keras.datasets import mnist

In [ ]:
!pip install scipy==1.1.0

In [ ]:
#================================================================================================
def load_generated_data(data_name):
    "load generated data"
    d=h5py.File(data_name)
    data=dict()
    data['F']=[]
    data['y']=[]
    data['F']=np.array(d['F']).T
    data['y']=np.array(d['y'])
    return data
def shuffle_in_unison_inplace(a, b):
    """Shuffle the arrays randomly"""
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
def data_label_shuffle(data_sets_org, percent_of_train, min_test_data=80, shuffle_data=True):
    """Divided the data to train and test and shuffle it"""
    perc = lambda i, t: np.rint((i * t) / 100).astype(np.int32)
    C = type('type_C', (object,), {})
    data_sets = C()
    stop_train_index = perc(percent_of_train[0], data_sets_org['F'].shape[0])
    start_test_index = stop_train_index
    if percent_of_train > min_test_data:
        start_test_index = perc(min_test_data, data_sets_org['F'].shape[0])
    data_sets.train = C()
    data_sets.test = C()
    if shuffle_data:
        shuffled_data, shuffled_labels = shuffle_in_unison_inplace(data_sets_org['F'], data_sets_org['y'])
    else:
        shuffled_data, shuffled_labels = data_sets_org.data, data_sets_org.labels
    data_sets.train.data = shuffled_data[:stop_train_index, :]
    data_sets.train.labels = shuffled_labels[:stop_train_index, :]
    data_sets.test.data = shuffled_data[start_test_index:, :]
    data_sets.test.labels = shuffled_labels[start_test_index:, :]
    return data_sets
def store_file(stringOfHistory,his):
    "Store the history of accuarcy"
    with open(stringOfHistory,'wb') as fp:
        return cPickle.dump(his,fp)
def load_DATA(name):
    "Load history"
    with open(name, 'rb') as f:
        data=cPickle.load(f)
        return data
#========================================================================================================================================================================================
class OutputObserver(keras.callbacks.Callback):
    'callback to observe the output of the network'
    def __init__(self, data_set,layernum,DATA,Label,bins,logs={}):
        self.out_log = dict()  # Save the output of each layer in each epoch
        self.out_log['eachlayer']=[]
        self.out_log['each_final_layer']=[]
        self.data_set = data_set
        self.layer_name='Layer'+str(layernum)
        self.DATA=DATA 
        self.Label=Label
        self.bins=bins
        self.local_IXT_cas_layer,self.local_ITY_cas_layer,self.finallocal_IXT_cas_layer,self.finallocal_ITY_cas_layer=[],[],[],[] # Save the information calculated by Binning method
        self.KDElocal_IXT_cas_layer,self.KDElocal_ITY_cas_layer,self.KDEfinallocal_IXT_cas_layer,self.KDEfinallocal_ITY_cas_layer=[],[],[],[] # Save the information calculated by PWD based upper bound method
        self.kdislocal_IXT_cas_layer,self.kdislocal_ITY_cas_layer,self.kdisfinallocal_IXT_cas_layer,self.kdisfinallocal_ITY_cas_layer=[],[],[],[]  # Save the information calculated by PWD based lower bound method
        self.pys1,self.unique_inverse_x,self. unique_inverse_y,self. pxs = extract_probs(self.Label,self.data_set) 
        self.PXs, self.PYs = np.asarray(self.pxs).T, np.asarray(self.pys1).T  # The distributation of Input and Label 
    def on_epoch_end(self, epoch, logs={}):
        "Calculate and save the information for each epoch"
        intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(self.layer_name).output) 
        layeroutput_cas=intermediate_layer_model.predict(self.data_set)                                           # Get the output of layer with name self.layer_name
        intermediate_predict_layer_model = Model(inputs=model.input,outputs=model.get_layer('Prediction_Layer').output) 
        finallayeroutput_cas=intermediate_predict_layer_model.predict(self.data_set)                              # Get the output of the classifier which is following the layer with name self.layer_name
        self.out_log['eachlayer'].append(layeroutput_cas)
        self.out_log['each_final_layer'].append(finallayeroutput_cas)
#        ==============================================================
        IX_CAS2,IY_CAS2=calculate_lower_mutual_information(layeroutput_cas[0:len(self.DATA)],self.PXs, self.PYs,self.unique_inverse_x,self.unique_inverse_y) # Calculate information for cascaded layer by PWD based lower bound method
        finalIX_CAS2,finalIY_CAS2=calculate_lower_mutual_information(finallayeroutput_cas[0:len(self.DATA)],self.PXs, self.PYs,self.unique_inverse_x,self.unique_inverse_y) # Calculate information for the classifier following the cascaded layer
        print('KdisI(X;T) is:'+str(IX_CAS2),'KdisI(Y;T) is:'+str(IY_CAS2))
        print('KdisI(X;T) is:'+str(finalIX_CAS2),'KdisI(Y;T) is:'+str(finalIY_CAS2))
        self.kdislocal_IXT_cas_layer.append(IX_CAS2)
        self.kdislocal_ITY_cas_layer.append(IY_CAS2)
        self.kdisfinallocal_IXT_cas_layer.append(finalIX_CAS2)
        self.kdisfinallocal_ITY_cas_layer.append(finalIY_CAS2)
#        ==============================================================
        IX_CAS,IY_CAS=calculate_mutual_information(layeroutput_cas[0:len(self.DATA)],self.PXs, self.PYs,self.unique_inverse_x,self.unique_inverse_y) # Calculate information for cascaded layer  by PWD based upper bound method
        finalIX_CAS,finalIY_CAS=calculate_mutual_information(finallayeroutput_cas[0:len(self.DATA)],self.PXs, self.PYs,self.unique_inverse_x,self.unique_inverse_y) # Calculate information for the classifier following the cascaded layer  by PWD based upper bound method
        print('KDEI(X;T) is:',IX_CAS,'KDEI(Y;T) is:',IY_CAS)
        print('KDEI(X;T) is:',finalIX_CAS,'KDEI(Y;T) is:',finalIY_CAS)
        self.KDElocal_IXT_cas_layer.append(IX_CAS)
        self.KDElocal_ITY_cas_layer.append(IY_CAS)
        self.KDEfinallocal_IXT_cas_layer.append(finalIX_CAS)
        self.KDEfinallocal_ITY_cas_layer.append(finalIY_CAS)
        
#        ================================================================================
        IX_CAS1,IY_CAS1=calculate_layer_mutual_information(layeroutput_cas[0:len(self.DATA)],self.PXs, self.PYs,self.unique_inverse_x,self.unique_inverse_y,self.bins) # Calculate information for cascaded layer by Binning method
        finalIX_CAS1,finalIY_CAS1=calculate_layer_mutual_information(finallayeroutput_cas[0:len(self.DATA)],self.PXs, self.PYs,self.unique_inverse_x,self.unique_inverse_y,self.bins) # Calculate information for the classifier following the cascaded layer by Binning method
        self.local_IXT_cas_layer.append(IX_CAS1)
        self.local_ITY_cas_layer.append(IY_CAS1)
        self.finallocal_IXT_cas_layer.append(finalIX_CAS1)
        self.finallocal_ITY_cas_layer.append(finalIY_CAS1)
#        ==============================================================

#        self.out_log.append(layeroutput_cas)
"The functions needed for creating network"
def connectOutputBlock(modelToConnectOut,outputsize,activation):
    "The final prediction block"
    modelToConnectOut.add(Dense(outputsize,activation=activation,name='Prediction_Layer'))
    return modelToConnectOut
def initialblock(structure_nodes,model,X_train,activation,layernum):
    "The initialization setting of the network"
    model.add(Dense(structure_nodes,activation=activation,input_dim=X_train.shape[1],kernel_regularizer=keras.regularizers.l1_l2(l1=0.01, l2=0.01),kernel_initializer=keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None),name='Layer'+str(layernum)))
    return model

def trainModel(modelToTrain,data,currentEpochs,batch_size,optimizer,accuarcy_name,filepath,layernum,DATA,Label):
    modelToTrain.compile(loss='mean_squared_error',optimizer=optimizer,metrics=[accuarcy_name])
    bins = np.linspace(-1, 1, 25)
    bins = bins.astype(np.float32)
    prediction_logs=OutputObserver(np.concatenate((data[0],data[2])),layernum,DATA,np.concatenate((data[1],data[3])),bins)
    trainingResults = modelToTrain.fit(data[0],data[1], batch_size=batch_size, epochs=currentEpochs, validation_data=(data[2],data[3]),verbose=1,callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1, mode='auto'),keras.callbacks.ModelCheckpoint(filepath+"/weights-improvement-{epoch:02d}.tf", monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1),prediction_logs])
    hist = trainingResults.history#keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=nb_epoch/10, verbose=1, mode='auto'),keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto'),
    return hist,prediction_logs.out_log,prediction_logs.local_IXT_cas_layer,prediction_logs.local_ITY_cas_layer,prediction_logs.finallocal_IXT_cas_layer,prediction_logs.finallocal_ITY_cas_layer,prediction_logs
#==================================================================The defination of the parameters we need==============================================================================
def parameters(number_of_data,num_of_train):
      "the structure can alter, e.g:list(map(int,np.linspace(2,100,20))) or [100,50,30,20,15,10]"
      input_dimension=12
      percent_of_train=np.array([70.00])
      structure=[15,10,7,3]# The structure  of the network (The number of nodes in each layer)
      outputsize=1          # The dimension of the prediction layer
      num_of_experment=str(structure)+'RULar_INFORMATION'+str(percent_of_train)+'_experment_'+str(number_of_data)+'_'+str(num_of_train)  #for change the saving name of results
      name_num_of_experment=str(structure)+'_experment_'+str(number_of_data)
      outputsize=1 # The size of the output
      lr = 0.001  #learning rate
      optimizer=Adam(lr=lr) # The name of the optimizer
      batch_size=256# The siz of the batch_size
      nb_epoch=100 # The number of training epo
      activation='tanh'#The activation function
      namecase=num_of_experment+'layer_output_case'
      accuarcy_name='acc'
      use_generated_structure=False
      return lr,optimizer,batch_size,nb_epoch,activation,namecase,structure,use_generated_structure,num_of_experment,outputsize,input_dimension,percent_of_train,accuarcy_name,name_num_of_experment

In [ ]:
for number_of_data in  ['MNIST']:#['Epileptic']:#,'Epileptic','gisette','LSVT','madelon','arcene','dexter'
    for num_of_train in range(1):
        lr,optimizer,batch_size,nb_epoch,activation,namecase,structure,use_generated_structure,num_of_experment,outputsize,input_dimension,percent_of_train,accuarcy_name,name_num_of_experment=parameters(number_of_data,num_of_train)
        if number_of_data=='MNIST':
          (X_train1, Y_train1),(X_test1, Y_test1)=mnist.load_data()
          X_train1 = np.array(X_train1.reshape(-1,X_train1.shape[1]*X_train1.shape[2]))
          X_test1= np.array(X_test1.reshape(-1,X_test1.shape[1]*X_test1.shape[2]))
          Y_train1=Y_train1.reshape(-1,1)
          Y_test1=Y_test1.reshape(-1,1)
          index_train1=np.where(Y_train1==5)
          index_train2=np.where(Y_train1==6)
          index_test1=np.where(Y_test1==5) 
          index_test2=np.where(Y_test1==6)
          Y_train1[index_train1]=0
          Y_train1[index_train2]=1
          Y_test1[index_test1]=0
          Y_test1[index_test2]=1
          # print('the task is binary classification:',X_train1[index_train1[0]].shape)
          X_train=np.concatenate((X_train1[index_train1[0]],X_train1[index_train2[0]]),axis=0)
          Y_train=np.concatenate((Y_train1[index_train1[0]],Y_train1[index_train2[0]]))
          X_test=np.concatenate((X_test1[index_test1[0]],X_test1[index_test2[0]]),axis=0)
          Y_test=np.concatenate((Y_test1[index_test1[0]],Y_test1[index_test2[0]]))
          # print('the task is binary classification:',.shape)
          # print(Y_test.shape)
          Data=np.concatenate((X_train,X_test))
          DATA=Data
          Label=np.concatenate((Y_train,Y_test))
          Label=Label.reshape(-1,1)
          print('the data set shape',Data.shape,len(index_train1[0])+len(index_test1[0]))
        else:
          Data_set=load_generated_data('dataset/'+number_of_data+'.mat')
          Data,Label=Data_set['F'],Data_set['y']
          DATA=Data
          Data_set_shuffled=data_label_shuffle(Data_set, percent_of_train,shuffle_data=True)
          X_train, Y_train,X_test, Y_test=Data_set_shuffled.train.data,Data_set_shuffled.train.labels,Data_set_shuffled.test.data,Data_set_shuffled.test.labels
          #=============================================================================================================
        #                                        Cascade Training process
        #=============================================================================================================

#        ---------------------------------------------------
        print('TRAINING Cascade learning')
        history = dict()
        output_history=dict()
        new_data_set=[X_train, Y_train,X_test, Y_test]
        # ==================================================================================
        sleep(0.1)
        #====================================================================training process==========================================================
        local_IXT_cas, local_ITY_cas,finallocal_IXT_cas, finallocal_ITY_cas=[],[],[],[]
        KDElocal_IXT_cas, KDElocal_ITY_cas,KDEfinallocal_IXT_cas, KDEfinallocal_ITY_cas=[],[],[],[]
        Kdislocal_IXT_cas, Kdislocal_ITY_cas,Kdisfinallocal_IXT_cas, Kdisfinallocal_ITY_cas=[],[],[],[]
        for layernum in tqdm.tqdm(range(len(structure))):
              if layernum==0:
                  model = Sequential()
                  model=initialblock(structure[layernum],model,X_train,activation,layernum)
                  model = connectOutputBlock(model,outputsize,activation)
                  model.summary()
                  filepath='check_point_model_layer_save/model_'+num_of_experment+'/layer_'+str(layernum) # The path of saving model
                  if not os.path.exists(filepath):
                        os.makedirs(filepath)
                  history['iter'+str(layernum)] = []
                  output_history['iter'+str(layernum)] = []
                  hist,prediction_logs_out,local_IXT_cas_layer, local_ITY_cas_layer,finallocal_IXT_cas_layer, finallocal_ITY_cas_layer,prediction_logs=trainModel(model,new_data_set,nb_epoch,batch_size,optimizer,accuarcy_name,filepath,layernum,DATA,Label)
                  history['iter'+str(layernum)].append(hist)
                  output_history['iter'+str(layernum)].append(prediction_logs_out)
                  X_train1,X_test1=X_train,X_test
                  model_layer_path='model_layer_save/model_'+num_of_experment # The path of saving model
                  if not os.path.exists(model_layer_path):
                      os.makedirs(model_layer_path)
                  model.save(model_layer_path+'/'+str(layernum)+'model.tf')
                  print('layer_'+str(layernum)+'_model is generated')
                  local_IXT_cas.append(local_IXT_cas_layer)
                  local_ITY_cas.append(local_ITY_cas_layer)
                  finallocal_IXT_cas.append(finallocal_IXT_cas_layer)
                  finallocal_ITY_cas.append(finallocal_ITY_cas_layer)
                  # ========================================
                  KDElocal_IXT_cas.append(prediction_logs.KDElocal_IXT_cas_layer)
                  KDElocal_ITY_cas.append(prediction_logs.KDElocal_ITY_cas_layer)
                  KDEfinallocal_IXT_cas.append(prediction_logs.KDEfinallocal_IXT_cas_layer)
                  KDEfinallocal_ITY_cas.append(prediction_logs.KDEfinallocal_ITY_cas_layer)
#                  =============================================
                  Kdislocal_IXT_cas.append(prediction_logs.kdislocal_IXT_cas_layer)
                  Kdislocal_ITY_cas.append(prediction_logs.kdislocal_ITY_cas_layer)
                  Kdisfinallocal_IXT_cas.append(prediction_logs.kdisfinallocal_IXT_cas_layer)
                  Kdisfinallocal_ITY_cas.append(prediction_logs.kdisfinallocal_ITY_cas_layer)
              else:
                  intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer('Layer'+str(layernum-1)).output)
                  prediction_class_train_updata=intermediate_layer_model.predict(X_train1)
                  prediction_class_test_updata=intermediate_layer_model.predict(X_test1)
                  new_data_set=[prediction_class_train_updata, Y_train,prediction_class_test_updata, Y_test]
                  del model,intermediate_layer_model
                  model= Sequential()
                  model=initialblock(structure[layernum],model,prediction_class_train_updata,activation,layernum)
                  model = connectOutputBlock(model,outputsize,activation)
                  model.summary()
                  filepath='check_point_model_layer_save/model_'+num_of_experment+'/layer_'+str(layernum) # The path of saving model
                  if not os.path.exists(filepath):
                        os.makedirs(filepath)
                  history['iter'+str(layernum)] = []
                  output_history['iter'+str(layernum)] = []
                  hist,prediction_logs_out,local_IXT_cas_layer, local_ITY_cas_layer,finallocal_IXT_cas_layer,finallocal_ITY_cas_layer,prediction_logs=trainModel(model,new_data_set,nb_epoch*(layernum+1),batch_size,optimizer,accuarcy_name,filepath,layernum,DATA,Label)
                  history['iter'+str(layernum)].append(hist)
                  output_history['iter'+str(layernum)].append(prediction_logs_out)
                  X_train1,X_test1=new_data_set[0],new_data_set[2]
                  model_layer_path='model_layer_save/model_'+num_of_experment # The path of saving model
                  if not os.path.exists(model_layer_path):
                    os.makedirs(model_layer_path)
                  model.save(model_layer_path+'/'+str(layernum)+'model.tf')
                  print('layer_'+str(layernum)+'_model is generated')
#                  =========================================================
                  local_IXT_cas.append(local_IXT_cas_layer)
                  local_ITY_cas.append(local_ITY_cas_layer)
                  finallocal_IXT_cas.append(finallocal_IXT_cas_layer)
                  finallocal_ITY_cas.append(finallocal_ITY_cas_layer)
#                  ===============================================================
                  KDElocal_IXT_cas.append(prediction_logs.KDElocal_IXT_cas_layer)
                  KDElocal_ITY_cas.append(prediction_logs.KDElocal_ITY_cas_layer)
                  KDEfinallocal_IXT_cas.append(prediction_logs.KDEfinallocal_IXT_cas_layer)
                  KDEfinallocal_ITY_cas.append(prediction_logs.KDEfinallocal_ITY_cas_layer)
#                  =============================================
                  Kdislocal_IXT_cas.append(prediction_logs.kdislocal_IXT_cas_layer)
                  Kdislocal_ITY_cas.append(prediction_logs.kdislocal_ITY_cas_layer)
                  Kdisfinallocal_IXT_cas.append(prediction_logs.kdisfinallocal_IXT_cas_layer)
                  Kdisfinallocal_ITY_cas.append(prediction_logs.kdisfinallocal_ITY_cas_layer)
#                  ================================================
        information_save_path='cascade_information_save/'+num_of_experment
        if not os.path.exists(information_save_path):
            os.makedirs(information_save_path)
        store_file(information_save_path+'/information_x.pkl',local_IXT_cas)
        store_file(information_save_path+'/information_y.pkl',local_ITY_cas)
        store_file(information_save_path+'/final_information_x.pkl',finallocal_IXT_cas)
        store_file(information_save_path+'/final_information_y.pkl',finallocal_ITY_cas)
#        =================================================================
        # with sess.as_default():
        #      KDElocal_IXT_cas=KDElocal_IXT_cas.eval()
        #      KDElocal_ITY_cas=KDElocal_ITY_cas.eval()
        #      KDEfinallocal_IXT_cas=KDEfinallocal_IXT_cas.eval()
        #      KDEfinallocal_ITY_cas=KDEfinallocal_ITY_cas.eval()
        # print('KDEI(X;T) is:',IX_CAS,'KDEI(Y;T) is:',IY_CAS)
        # print('KDEI(X;T) is:',finalIX_CAS,'KDEI(Y;T) is:',finalIY_CAS)
        store_file(information_save_path+'/KDEinformation_x.pkl',KDElocal_IXT_cas)
        store_file(information_save_path+'/KDEinformation_y.pkl',KDElocal_ITY_cas)
        store_file(information_save_path+'/KDEfinal_information_x.pkl',KDEfinallocal_IXT_cas)
        store_file(information_save_path+'/KDEfinal_information_y.pkl',KDEfinallocal_ITY_cas)
#        ======================================================================
        store_file(information_save_path+'/Kdisinformation_x.pkl',Kdislocal_IXT_cas)
        store_file(information_save_path+'/Kdisinformation_y.pkl',Kdislocal_ITY_cas)
        store_file(information_save_path+'/Kdisfinal_information_x.pkl',Kdisfinallocal_IXT_cas)
        store_file(information_save_path+'/Kdisfinal_information_y.pkl',Kdisfinallocal_ITY_cas)
#        =========================================================================

        #========================================================plot training process=======================================================
        accuarcy_save_path='history_accuarcy/acc_'+num_of_experment
        if not os.path.exists(accuarcy_save_path):
            os.makedirs(accuarcy_save_path)
        store_file(accuarcy_save_path+'/'+'acc.pkl',history)
        store_file(accuarcy_save_path+'/'+'output_history.pkl',output_history)
        print('history is saved')
       #=====================================================================================================================================
        HISTORY=load_DATA(accuarcy_save_path+'/'+'acc.pkl')
        hISTORY=dict()
        for layernum in range(len(structure)):
           hISTORY['iter_TRAIN'+str(layernum)]=[]
           hISTORY['iter_TEST'+str(layernum)]=[]
           for i in range(len(HISTORY['iter'+str(layernum)])):
                 hISTORY['iter_TRAIN'+str(layernum)].append(HISTORY['iter'+str(layernum)][i][accuarcy_name])
                 hISTORY['iter_TEST'+str(layernum)].append(HISTORY['iter'+str(layernum)][i]['val_'+accuarcy_name])
       #--------------------------------------------------------------------------------------------------------------------------------------
        accuarcy_image_save_path='image_history_accuarcy/acc_'+num_of_experment
        if not os.path.exists(accuarcy_image_save_path):
           os.makedirs(accuarcy_image_save_path)
        fig_all=plt.figure(figsize=(15,8))#figsize=(10, 10)
        axis_font=28
        axes=fig_all.add_subplot(1,2,1)
        for layernum in range(len(structure)):
           axes.plot(hISTORY['iter_TRAIN'+str(layernum)][0],label='CAS_train_acc_layer'+str(layernum))
           axes.text(0.5*len(hISTORY['iter_TRAIN'+str(0)][0]),0.5+0.1*layernum,'Layer_'+str(layernum)+'_accuarcy is:'+str(hISTORY['iter_TRAIN'+str(layernum)][0][-1]))
        plt.legend()
        # plt.grid(True)
        axes.set_xlim(-0.1)
        axes.set_ylim(-0.1)
        axes.set_yticks(np.linspace(0,1,11))
        plt.title('ACC_TRAIN',fontsize=axis_font+2)
        plt.xlabel('Epoch',fontsize=axis_font)
        plt.ylabel('ACC',fontsize=axis_font)
        axes=fig_all.add_subplot(1,2,2)
        for layernum in range(len(structure)):
           axes.plot(hISTORY['iter_TEST'+str(layernum)][0],label='CAS_test_acc_layer'+str(layernum))
           axes.text(0.5*len(hISTORY['iter_TRAIN'+str(0)][0]),0.5+0.1*layernum,'Layer_'+str(layernum)+'_accuarcy is:'+str(hISTORY['iter_TEST'+str(layernum)][0][-1]))
        plt.legend()
        # plt.grid(True)
        axes.set_xlim(-0.1)
        axes.set_ylim(-0.1)
        axes.set_yticks(np.linspace(0,1,11))
        plt.title('ACC_TEST',fontsize=axis_font+2)
        plt.xlabel('Epoch',fontsize=axis_font)
        plt.ylabel('ACC',fontsize=axis_font)
        plt.suptitle('all in one'+num_of_experment)
       #plt.show()
        fig_all.savefig(accuarcy_image_save_path+'/accuarcy.png')
        fig_all.savefig('image_history_accuarcy/accuarcy'+num_of_experment+'.png')
       # print(output_history)


#Plot

In [ ]:
!cp /content/model_layer_save -r drive/MyDrive/Cascade/tanh/
!cp /content/check_point_model_layer_save -r drive/MyDrive/Cascade/tanh/

In [ ]:
!cp drive/MyDrive/Cascade/tanh/ -r /content/model_layer_save
!cp  drive/MyDrive/Cascade/tanh/ -r /content/check_point_model_layer_save 

In [ ]:
!cp drive/MyDrive/Cascade/plot.py /content/

In [ ]:
from __future__ import print_function
import pickle as cPickle
import numpy as np
import plot as plo
import matplotlib.pyplot as plt
import os
from training_parameters import parameters # Import parametrs used for training model

In [ ]:
def load_DATA(name):
    "Load saved data"
    with open(name, 'rb') as f:
        data=cPickle.load(f)
        return data 

def parameters(number_of_data,num_of_train):
      "the structure can alter, e.g:list(map(int,np.linspace(2,100,20))) or [100,50,30,20,15,10]"
#      input_dimension=12
      percent_of_train=np.array([70.00])
      structure=[20,10,7,3]#[20,10,7,3]# The structure  of the network (The number of nodes in each layer)
      outputsize=1          # The dimension of the prediction layer
      num_of_experment='check_weight_no_stop'+str(structure)+'RULar_INFORMATION'+str(percent_of_train)+'_'+'experment_'+str(number_of_data)+'_'+str(num_of_train)#'test_INFORMATION_version_'+str(percent_of_train)+'_'+str(structure)+'_full_size_experment_'+str(number_of_data)+'_'+str(num_of_train)#str(structure)+'RULar_INFORMATION'+str(percent_of_train)+'_'+'experment_'+str(number_of_data)+'_'+str(num_of_train)#for change the saving name of results#'structure_test_cascade_learning'+str(percent_of_train)+'_'+str(structure)+'_'+'experment_'+str(number_of_data)+'_'+str(num_of_train)#'test_KDE_save_cascade_learning'+str(percent_of_train)+'_'+str(structure)+'_experment_'+str(number_of_data)+'_'+str(num_of_train)   #str(structure)+'RULar_INFORMATION'+str(percent_of_train)+'_'+'experment_'+str(number_of_data)+'_'+str(num_of_train)#for change the saving name of results
      name_num_of_experment=str(structure)+'_experment_'+str(number_of_data)
      outputsize=1 # The size of the output
      nb_epoch=3000 # The number of training epoch
      num_of_bins=25
      bins = np.linspace(-1, 1, num_of_bins)
      bins = bins.astype(np.float32)
      accuarcy_name='acc'
      return nb_epoch,structure,num_of_experment,outputsize,percent_of_train,name_num_of_experment,bins,accuarcy_name

xticks = np.linspace(0,10,11)
yticks = np.linspace(0,0.5,6)
x_lim=-0
y_lim=-0
axis_font=25
fig_size = (16,8)
font_size = 25    
#======================================================================
def plot_layer_sepreate(local_IXT_CAS,local_ITY_CAS,finallocal_IXT_CAS,finallocal_ITY_CAS,structure,num_of_experment):
        fig1=plt.figure(figsize=(15, 8))
        local_IXT_CAS_array,local_ITY_CAS_array=np.array(local_IXT_CAS),np.array(local_ITY_CAS)
        finallocal_IXT_CAS_array,finallocal_ITY_CAS_array=np.array(finallocal_IXT_CAS),np.array(finallocal_ITY_CAS)
        for layernum in range(len(structure)):
            ax=fig1.add_subplot(2,np.ceil(len(structure)/2),layernum+1)
            ax.scatter(local_IXT_CAS_array[layernum],local_ITY_CAS_array[layernum],label='CAS_layer'+str(layernum))
            print('the epoch is:'+str(len(local_IXT_CAS_array[layernum])))
            ax.scatter(local_IXT_CAS_array[layernum][-1],local_ITY_CAS_array[layernum][-1],label='CAS_layer'+str(layernum)+'final_epoch',facecolors='none', edgecolors='b')
            ax.scatter(finallocal_IXT_CAS_array[layernum],finallocal_ITY_CAS_array[layernum],label='finalCAS_layer'+str(layernum)) 
            ax.scatter(finallocal_IXT_CAS_array[layernum][-1],finallocal_ITY_CAS_array[layernum][-1],label='finalCAS_layer'+str(layernum)+'final_epoch',facecolors='none', edgecolors='g') 
            fig1.suptitle('information_comparison_'+num_of_experment,fontsize=axis_font + 2)
            ax.set_xticks(xticks)
            ax.set_yticks(yticks)
            ax.set_xlim(x_lim)
            ax.set_ylim(y_lim)
            plt.xlabel('$I(X;T)$',fontsize=axis_font)
            plt.ylabel('$I(Y;T)$',fontsize=axis_font)
            plt.legend()
        # plt.show()
        information_figure_path='Cascade_information_result_saving/'+num_of_experment
        finalinformation_figure_path='Cascade_final_information_result_saving/'+num_of_experment
        if not os.path.exists(information_figure_path):
            os.makedirs(information_figure_path)
        if not os.path.exists(finalinformation_figure_path):
            os.makedirs(finalinformation_figure_path)
        fig1.savefig(information_figure_path+'/'+'hidden_final_compare.png')



In [ ]:
for number_of_data in ['MNIST']:#Epileptic,gisette
    for num_of_train in range(1):
        nb_epoch,structure,num_of_experment,outputsize,percent_of_train,name_num_of_experment,bins,accuarcy_name=parameters(number_of_data,num_of_train)
        
        ###===========================plot training perfprmance========================
        accuarcy_save_path='history_accuarcy/acc_'+num_of_experment
        HISTORY=load_DATA(accuarcy_save_path+'/acc.pkl')
        max_epoch=np.array([len(HISTORY['iter'+str(layernum)][0][accuarcy_name]) for layernum in range(len(structure))]).max()
        print('The maximium epoch is:',str(max_epoch))

        epochsInds=np.linspace(1,max_epoch,max_epoch)
        hISTORY=dict()
        for layernum in range(len(structure)):
            hISTORY['iter_TRAIN'+str(layernum)]=[]
            hISTORY['iter_TEST'+str(layernum)]=[]
            for i in range(len(HISTORY['iter'+str(layernum)])):
                  hISTORY['iter_TRAIN'+str(layernum)].append(HISTORY['iter'+str(layernum)][i][accuarcy_name]) 
                  hISTORY['iter_TEST'+str(layernum)].append(HISTORY['iter'+str(layernum)][i]['val_'+accuarcy_name])
        #--------------------------------------------------------------------------------------------------------------------------------------
        accuarcy_image_save_path='image_history_accuarcy/acc_'+num_of_experment
        if not os.path.exists(accuarcy_image_save_path):
            os.makedirs(accuarcy_image_save_path)
        fig_all=plt.figure(figsize=(16,10))
        axis_font=28
        axes=fig_all.add_subplot(1,2,1)
        for layernum in range(len(structure)):
            axes.plot(hISTORY['iter_TRAIN'+str(layernum)][0],label='Layer '+str(layernum)+', Average:'+str(str(round(np.mean(hISTORY['iter_TRAIN'+str(layernum)][0][-100:-1]),3))))
            # axes.text(200,0.9+0.02*layernum,'Layer '+str(layernum)+':'+str(round(np.mean(hISTORY['iter_TRAIN'+str(layernum)][0][-100:-1]),3)),fontsize=axis_font-6)#0.5*len(hISTORY['iter_TRAIN'+str(layernum)][0])
        plt.legend(fontsize=axis_font-10,loc=1,ncol=1,bbox_to_anchor=(0.98,0.75))
        # plt.grid(True)
        axes.set_xlim(-0.1)
        axes.set_ylim(0.4)
        axes.set_yticks(np.linspace(0.4,1.1,7)) 
        # plt.title('ACC_TRAIN',fontsize=axis_font+2)
        plt.xlabel('Epoch\n (a) Training accuarcy',fontsize=axis_font)
        plt.ylabel('Accuarcy',fontsize=axis_font)
        plt.xticks(rotation=45,fontsize=22)
        plt.yticks(fontsize=22)
        # ===================================================================
        rect = [0.36,0.12,0.6,0.4]
        box = axes.get_position()
        width = box.width
        height = box.height
        inax_position  = axes.transAxes.transform(rect[0:2])
        transFigure = fig_all.transFigure.inverted()
        infig_position = transFigure.transform(inax_position)    
        xl = infig_position[0]
        yl = infig_position[1]
        width *= rect[2]
        height *= rect[3]  # <= Typo was here
        ax2= fig_all.add_axes([xl,yl,width,height])
        for layernum in range(len(structure)):
            ax2.plot(hISTORY['iter_TRAIN'+str(layernum)][0][0:50],label='Layer '+str(layernum))
        plt.xlabel('Epoch',fontsize=axis_font-2)
        plt.ylabel('Accuarcy',fontsize=axis_font-2)
        plt.xticks(fontsize=22)
        plt.yticks(fontsize=22)
        plt.legend(fontsize=axis_font-10,loc='3')
        # ======================================================================
        axes1=fig_all.add_subplot(1,2,2) 
        for layernum in range(len(structure)):
            axes1.plot(hISTORY['iter_TEST'+str(layernum)][0],label='Layer '+str(layernum)+', Average:'+str(str(round(np.mean(hISTORY['iter_TEST'+str(layernum)][0][-100:-1]),3))))
            # axes1.text(200,0.9+0.02*layernum,'Layer '+str(layernum)+':'+str(round(np.mean(hISTORY['iter_TEST'+str(layernum)][0][-100:-1]),3)),fontsize=axis_font-6)
        plt.legend(fontsize=axis_font-10,loc=1,ncol=1,bbox_to_anchor=(0.98,0.75))
        # plt.grid(True)
        axes1.set_xlim(-0.1)
        axes1.set_ylim(0.4)
        axes1.set_yticks(np.linspace(0.4,1.1,7))
        # plt.title('ACC_TEST',fontsize=axis_font+2)
        plt.xlabel('Epoch\n (b) Testing accuarcy',fontsize=axis_font)
        # plt.ylabel('Accuarcy',fontsize=axis_font)
        plt.xticks(rotation=45,fontsize=22)
        plt.yticks(fontsize=22)
        # ===========================================================================
        rect = [0.36,0.12,0.6,0.4]
        box = axes1.get_position()
        width = box.width
        height = box.height
        inax_position  = axes1.transAxes.transform(rect[0:2])
        transFigure = fig_all.transFigure.inverted()
        infig_position = transFigure.transform(inax_position)    
        xl = infig_position[0]
        yl = infig_position[1]
        width *= rect[2]
        height *= rect[3]  # <= Typo was here
        ax1= fig_all.add_axes([xl,yl,width,height])
        for layernum in range(len(structure)):
            ax1.plot(hISTORY['iter_TEST'+str(layernum)][0][0:50],label='Layer '+str(layernum))
        plt.xlabel('Epoch',fontsize=axis_font-2)
        plt.ylabel('Accuarcy',fontsize=axis_font-2)
        plt.xticks(fontsize=22)
        plt.yticks(fontsize=22)
        plt.legend(fontsize=axis_font-10,loc='3')
        plt.show()
        # ==========================================================================
        # plt.suptitle('all in one'+num_of_experment)
        #plt.show()
        fig_all.savefig(accuarcy_image_save_path+'/accuarcy.png',bbox_inches='tight')
        fig_all.savefig('image_history_accuarcy/accuarcy'+num_of_experment+'.png',bbox_inches='tight')

        local_IXT_CAS=load_DATA(information_save_path+'/KDEinformation_x.pkl')
        local_ITY_CAS=load_DATA(information_save_path+'/KDEinformation_y.pkl')
        finallocal_IXT_CAS=load_DATA(information_save_path+'/KDEfinal_information_x.pkl')
        finallocal_ITY_CAS=load_DATA(information_save_path+'/KDEfinal_information_y.pkl')
        #==============================plot final============================================
        information_figure_path='Cascade_information_result_saving/KDE'+num_of_experment
        finalinformation_figure_path='Cascade_final_information_result_saving/KDE'+num_of_experment
        if not os.path.exists(information_figure_path):
            os.makedirs(information_figure_path) 
        if not os.path.exists(finalinformation_figure_path):
            os.makedirs(finalinformation_figure_path)
        plot_layer_sepreate(local_IXT_CAS,local_ITY_CAS,finallocal_IXT_CAS,finallocal_ITY_CAS,structure,'KDE'+num_of_experment)
        local_IXT_CAS_array,local_ITY_CAS_array=np.array(local_IXT_CAS),np.array(local_ITY_CAS)
        finallocal_IXT_CAS_array,finallocal_ITY_CAS_array=np.array(finallocal_IXT_CAS),np.array(finallocal_ITY_CAS)
        plo.plotfigure(local_IXT_CAS_array,local_ITY_CAS_array,epochsInds,information_figure_path)
        plo.plotfigure(finallocal_IXT_CAS_array,finallocal_ITY_CAS_array,epochsInds,finalinformation_figure_path)
        

In [ ]:
sm = plt.cm.ScalarMappable(cmap='gnuplot', norm=plt.Normalize(vmin=0, vmax=100))
sm._A = []
PLOT_LAYERS    = None 
num_layers = 5

activations = ['tanh', 'relu']

if PLOT_LAYERS is None:
        PLOT_LAYERS = []
        for lndx in range(num_layers):
            #if d['data']['activity_tst'][lndx].shape[1] < 200 and lndx != num_layers - 1:
            PLOT_LAYERS.append(lndx)
fig=plt.figure(figsize=(5,5))


    
for epoch in range(1, epochsInds.shape[0]):
    c = sm.to_rgba(epoch)
    xmvals = local_IXT_CAS_array[:,epoch]
    ymvals = local_ITY_CAS_array[:, epoch]
    plt.plot(xmvals, ymvals, c=c, alpha=0.1, zorder=1)
    plt.scatter(xmvals, ymvals, s=20, facecolors=[c for _ in PLOT_LAYERS], edgecolor='none', zorder=2)


plt.ylim([0, 1])
plt.xlim([0, 12])
#     plt.ylim([0, 3.5])
#     plt.xlim([0, 14])
plt.xlabel('I(X;M)')
plt.ylabel('I(Y;M)')
plt.title('tanh')
    
cbaxes = fig.add_axes([1.0, 0.125, 0.03, 0.8]) 
plt.colorbar(sm, label='Epoch', cax=cbaxes)

DO_SAVE = True

!mkdir /content/plots
if DO_SAVE:
    plt.savefig('plots/' + 'Synthetic tanh batch')
!cp /content/plots -r drive/MyDrive/Cascade/tanh